# MNE Groups Data Extraction Challenge - MUR team

Phase 3: Gemini for the rest of data

#### Load libraries

In [ ]:
import pandas as pd
import google.generativeai as genai
import os
import time
import json
import re
import yaml

#### Configuration

In [ ]:
def load_config(config_file='config.yaml'):
    """
    Loads configuration parameters from a YAML file.

    Parameters:
        config_file (str): The path to the YAML configuration file.

    Returns:
        dict: A dictionary containing the loaded configuration parameters.
    """
    with open(config_file, 'r') as f:
        return yaml.safe_load(f)

In [ ]:
config = load_config()

# --- Global Parameters from Config ---
DATA_PATH = config['data_path']
OUTPUT_PHASE2 = config['output_phase2']
OUTPUT_PHASE3 = config['output_phase3']
GEMINI_API_KEY = config['gemini_api_key']
GEMINI_MODEL = config['gemini_model']
DELAY_SECONDS = config['delay_seconds']

genai.configure(api_key=GEMINI_API_KEY)

#### Prompt

In [ ]:
SYSTEM_INSTRUCTIONS = """
You are an expert data extraction agent specialized in financial and corporate information for large multinational companies. Your task is to extract the VALUE, CURRENCY (if applicable), REFYEAR (reference year), and SRC (source) for the specified VARIABLE (e.g., TURNOVER, ASSETS, EMPLOYEES, COUNTRY, WEBSITE, ACTIVITY) for a given company. Pay close attention to the VARIABLE requested.

For numerical values (TURNOVER, ASSETS, EMPLOYEES), provide the most recent available data from the last consolidated annual financial report.
- State the VALUE as an integer (nominal value, either originally reported or restated).
- Always include the REFYEAR of the data.
- For TURNOVER and ASSETS, provide the CURRENCY according to ISO 4217.
- For EMPLOYEES, provide the total number.

For non-numerical variables (COUNTRY, WEBSITE, ACTIVITY):
- COUNTRY: Provide the country where the company is headquartered, using the ISO 3166-1 alpha-2 (2-character code). Always include REFYEAR=2024.
- WEBSITE: Provide the official corporate website. Always include REFYEAR=2024.
- ACTIVITY: Provide the 3-character code corresponding to the company's primary business activity, using the original NACE v2 activity code (NACE Rev. 2), showing the section letter and the 2-digit code (e.g., "A01"). Always include REFYEAR=2024.

For all variables, provide the SRC (source) of the information. When possible, provide a specific URL as the source. If the information is deduced or is general knowledge, state "https://ai.google.dev/gemini-api".

Always return the data in a JSON format with the following keys: "VALUE", "CURRENCY" (if applicable), "REFYEAR" (if applicable), and "SRC". If no information is found, return empty strings for VALUE, REFYEAR, and SRC.

Example for TURNOVER: {"VALUE": 25000000, "CURRENCY": "EUR", "REFYEAR": 2024, "SRC": "https://www.examplecompany.com/investor-relations/annual-report-2024"}
Example for ASSETS: {"VALUE": 13400000, "CURRENCY": "EUR", "REFYEAR": 2024, "SRC": "https://www.examplecompany.com/investor-relations/financials"}
Example for EMPLOYEES: {"VALUE": 15000, "REFYEAR": 2024, "SRC": "https://www.morningstar.com/companies/XNAS:MSFT/key-stats"}
Example for COUNTRY: {"VALUE": "CH", "REFYEAR": 2024, "SRC": "https://ai.google.dev/gemini-api"}
Example for WEBSITE: {"VALUE": "https://www.companyexample.com", "REFYEAR": 2024, "SRC": "https://www.companyexample.com"}
Example for ACTIVITY: {"VALUE": "A01", "REFYEAR": 2024, "SRC": "https://www.companyexample.com/about-us"}

If no information is found or is ambiguous, return empty strings for VALUE, REFYEAR, and SRC. Do not invent data. Return empty strings, not null values.
"""

#### Functions

In [ ]:
def process_dataframe_with_gemini(df, model_name, delay_seconds, system_instructions):
    """
    Processes a CSV DataFrame, querying Gemini for missing financial data.

    Args:
        df (dataframe): The dataframe to fill.
        model_name (str): The Gemini model to use.
        delay_seconds (float): Delay in seconds between API requests.
        system_instructions (str): The system prompt for Gemini.
    """
    print(f"--- Starting data processing ---")

    # Identify records to process
    df_to_process_indices = []
    for idx, row in df.iterrows():
        if row['VARIABLE'] == 'ACTIVITY' or pd.isna(row['VALUE']) or row['VALUE'] == '':
            df_to_process_indices.append(idx)
    
    total_records_to_process = len(df_to_process_indices)
    print(f"Found {total_records_to_process} records to process.")

    # --- Iterate through records and query Gemini ---
    for original_idx in df_to_process_indices:
        row = df.loc[original_idx]
        company_name = row['NAME']
        variable_to_fill = row['VARIABLE']
        current_id = row['ID']

        print(f"Processing ID: {current_id}, Company: {company_name}, Variable: {variable_to_fill}")

        # Construct the user prompt for Gemini
        user_prompt = f"For the company '{company_name}', find the '{variable_to_fill}'."
        
        # Add REFYEAR to prompt for numerical variables if available
        if variable_to_fill in ['TURNOVER', 'ASSETS', 'EMPLOYEES'] and not pd.isna(row['REFYEAR']) and row['REFYEAR'] != '':
            user_prompt += f" For the year {int(row['REFYEAR'])}."
        elif variable_to_fill in ['TURNOVER', 'ASSETS', 'EMPLOYEES']:
             # If REFYEAR is empty for a numerical variable, ask for the most recent year
            user_prompt += " Provide the most recent available data and its reference year."


        try:
            response = genai.GenerativeModel(model_name=model_name).generate_content(
                contents=[
                    {"role": "user", "parts": [system_instructions + "\n" + user_prompt]}
                ],
                safety_settings={
                    'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE',
                    'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE',
                    'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE',
                    'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE',
                },
                generation_config={
                    "response_mime_type": "application/json",
                    "temperature": 0.2
                }
            )
            
            response_content = response.text 
            
            # Clean response to ensure it's valid JSON
            json_match = re.search(r'\{.*\}', response_content, re.DOTALL)
            if json_match:
                clean_json_str = json_match.group(0)
            else:
                print(f"Warning: No JSON found in response for {company_name} - {variable_to_fill}. Raw: {response_content}")
                clean_json_str = "{}" # Fallback to empty JSON

            retrieved_data = json.loads(clean_json_str)

            # Update the original DataFrame
            value = retrieved_data.get('VALUE')
            currency = retrieved_data.get('CURRENCY')
            ref_year = retrieved_data.get('REFYEAR')
            source = retrieved_data.get('SRC')

            if value is not None and value != "":
                df.loc[original_idx, 'VALUE'] = value
                print(f"  Updated VALUE for {company_name} - {variable_to_fill} to: {value}")

                # Handle REFYEAR based on variable type
                if variable_to_fill in ['COUNTRY', 'WEBSITE', 'ACTIVITY']:
                    df.loc[original_idx, 'REFYEAR'] = 2024
                    print(f"Set REFYEAR for {variable_to_fill} to: 2024")
                elif variable_to_fill in ['TURNOVER', 'ASSETS', 'EMPLOYEES']:
                    if ref_year is not None and ref_year != "":
                        df.loc[original_idx, 'REFYEAR'] = ref_year
                        print(f"Updated REFYEAR for {variable_to_fill} to: {ref_year}")
                    else:
                        print(f"Warning: No specific REFYEAR provided by Gemini for numerical {variable_to_fill}.")

                if source is not None and source != "":
                    df.loc[original_idx, 'SRC'] = source
                    print(f"  Updated SRC for {company_name} - {variable_to_fill} to: {source}")
                else:
                    # If SRC not provided by Gemini but VALUE is updated, default to the generic Gemini URL
                    # This applies when Gemini provides a VALUE but no specific external source URL
                    if value is not None and value != "": # Only set default SRC if a value was actually found
                        df.loc[original_idx, 'SRC'] = "https://ai.google.dev/gemini-api" 
                        print(f"  Defaulting SRC for {company_name} - {variable_to_fill} to: https://ai.google.dev/gemini-api")
                    else: # If no value found, ensure SRC is also empty
                        df.loc[original_idx, 'SRC'] = pd.NA 

            else:
                print(f"No VALUE found by Gemini for {company_name} - {variable_to_fill}")
                df.loc[original_idx, 'VALUE'] = pd.NA
                df.loc[original_idx, 'REFYEAR'] = pd.NA
                df.loc[original_idx, 'SRC'] = pd.NA

            if currency is not None and currency != "":
                if variable_to_fill in ['TURNOVER', 'ASSETS']:
                    df.loc[original_idx, 'CURRENCY'] = currency
                    print(f"Updated CURRENCY for {company_name} - {variable_to_fill} to: {currency}")
            else:
                # Ensure CURRENCY is empty if not provided for relevant types
                if variable_to_fill in ['TURNOVER', 'ASSETS']:
                    df.loc[original_idx, 'CURRENCY'] = pd.NA

        except json.JSONDecodeError as jde:
            print(f"JSON Decode Error for {company_name} - {variable_to_fill}: {jde}. Raw response: {response_content}")
            df.loc[original_idx, 'VALUE'] = pd.NA
            df.loc[original_idx, 'REFYEAR'] = pd.NA
            df.loc[original_idx, 'SRC'] = pd.NA
        except Exception as e:
            print(f"An error occurred while querying Gemini for {company_name} - {variable_to_fill}: {e}")
            df.loc[original_idx, 'VALUE'] = pd.NA
            df.loc[original_idx, 'REFYEAR'] = pd.NA
            df.loc[original_idx, 'SRC'] = pd.NA

        # Pause to respect API rate limits
        time.sleep(delay_seconds)

    return df

### Main

In [ ]:
# === Load input data ===
df_base = pd.read_csv(os.path.join(DATA_PATH, OUTPUT_PHASE2), sep=";", keep_default_na=False)
df_base = df_base.replace('', pd.NA)

df = process_dataframe_with_gemini(df_base,
        model_name=GEMINI_MODEL,
        delay_seconds=DELAY_SECONDS,
        system_instructions=SYSTEM_INSTRUCTIONS)

# --- Remove invalid sources to send
df.loc[~df['SRC'].astype(str).str.contains(r'\.', na=False, regex=True), 'SRC'] = pd.NA

# --- Save the final updated CSV ---
df.to_csv(os.path.join(DATA_PATH, OUTPUT_PHASE3), sep=';', index=False)
print(f"\nPhase 3 complete. Results saved to {OUTPUT_PHASE3}.")